# Convolution Neural Net (CNN)
to detect apples

# Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cosmos_functions as cf

import pprint
import random
import torch
import torchvision
import torch.nn.functional as FF
import torchvision.transforms.functional as F
import torchvision.transforms as T
import wandb
from init.config import *
from datetime import datetime
from os import listdir, path
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split
from torchvision.io import ImageReadMode
from torchvision.io import read_image
from tqdm import tqdm

In [ ]:
x = userAccountID 

print(x)


In [ ]:
# set the device

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
x = torch.ones(1, device=device)

print(f"Device is '{device}' Thus a tensor will look like this: {x}")

In [ ]:
# function to show an image from the dataloader with its label
def show_batch(train_d,test_d,val_d):
    # Get a batch of data from the DataLoader
    data_train = next(iter(train_d))
    data_test = next(iter(test_d))
    data_val = next(iter(val_d))

    # Set the savefig.bbox parameter to 'tight'
    plt.rcParams["savefig.bbox"] = 'tight'

    # Retrieve the first tensor and its corresponding label
    image_train = data_train[0][0]
    image_test = data_test[0][0]
    image_val = data_val[0][0]
    label_train = data_train[1][0]
    label_test = data_test[1][0]
    label_val = data_val[1][0]
  

    # Create a figure with two subplots
    fig, axes = plt.subplots(1, 3)


    # Convert the image tensor to a NumPy array and transpose the dimensions
    np_image_train = image_train.permute(1, 2, 0).numpy()
    np_image_test = image_test.permute(1, 2, 0).numpy()
    np_image_val = image_val.permute(1, 2, 0).numpy()
    

        # Display the original image in the first subplot
    axes[0].imshow(np_image_train)  # ,cmap='gray' after the image name to create a greyscale image.
    axes[0].set_title(f'{label_train}, {image_train.shape}')

    # Display the pooled image in the second subplot
    axes[1].imshow(np_image_test)  # ,cmap='gray' after the image name to create a greyscale image.
    axes[1].set_title(f'{label_test}, {image_test.shape}')

    # Display the pooled image in the third subplot
    axes[2].imshow(np_image_val)  # ,cmap='gray' after the image name to create a greyscale image.
    axes[2].set_title(f'{label_val}, {image_val.shape}')

    # Adjust spacing between subplots to prevent overlap
    fig.tight_layout()

    # Show the plot
    plt.show()

In [ ]:
# function to determine accuracy 

def evaluate_accuracy(logits, y_true):
    # get index with the largest logit value PER OBSERVATION
    _, y_pred = torch.max(logits, dim=1)

    # calculate proportion of correct prediction
    correct_pred = (y_pred == y_true).float()  # PyCharm error Unresolved attribute reference 'float' for class 'bool'
    acc = correct_pred.sum() / len(correct_pred)

    return acc * 100

In [ ]:
# function to train the model

def train(model, train_loader, val_loader, criterion, optimizer, n_epochs, model_file_name='model.pt'):
    # initialize container variable for model performance results per epoch
    history = {
        'n_epochs': n_epochs,
        'loss': {
            'train': [],
            'val': []
        },
        'acc': {
            'train': [],
            'val': []
        }
    }

    # initialize tracker for minimum validation loss
    val_loss_min = np.Inf

    # loop per epoch
    for epoch in tqdm(range(n_epochs)):
        # initialize tracker for training performance
        train_acc = 0
        train_loss = 0

        ###################
        # train the model #
        ###################

        # prepare model for training
        model.train()

        # loop for each batch
        for data, target in tqdm(train_loader):
            # move data to device
            data, target = data.to(device), target.to(device)
            
            # STEP 1: clear gradients
            optimizer.zero_grad()
            # STEP 2: forward pass
            output = model(data)
            # STEP 3: calculate the loss
            loss = criterion(output, target)
            # STEP 4: backward pass
            loss.backward()
            # STEP 5: perform parameter update
            optimizer.step()
            # STEP 6: accumulate training loss and accuracy
            train_loss += loss.item() * data.size(0)
            acc = evaluate_accuracy(output, target)
            train_acc += acc.item() * data.size(0)

        ######################
        # validate the model #
        ######################

        # disable gradient calculation
        with torch.no_grad():
            # initialize tracker for validation performance
            val_acc = 0
            val_loss = 0

            # prepare model for evaluation
            model.eval()

            # loop for each batch
            for data, target in tqdm(val_loader):
                # move data to device
                data, target = data.to(device), target.to(device)
                # STEP 1: forward pass
                output = model(data)
                # STEP 2: calculate the loss
                loss = criterion(output, target)
                # STEP 3: accumulate validation loss and accuracy
                val_loss += loss.item() * data.size(0)
                acc = evaluate_accuracy(output, target)
                val_acc += acc.item() * data.size(0)

        ####################
        # model evaluation #
        ####################

        # calculate average loss over an epoch
        train_loss /= len(train_loader.sampler)
        val_loss /= len(val_loader.sampler)
        history['loss']['train'].append(train_loss)
        history['loss']['val'].append(val_loss)

        # calculate average accuracy over an epoch
        train_acc /= len(train_loader.sampler)
        val_acc /= len(val_loader.sampler)
        history['acc']['train'].append(train_acc)
        history['acc']['val'].append(val_acc)

        # print training progress per epoch
        print(f'Epoch {epoch+1:03} | Train Loss: {train_loss:.5f} | Val Loss: {val_loss:.5f} | Train Acc: {train_acc:.2f} | Val Acc: {val_acc:.2f}')

        # save model if validation loss has decreased
        if val_loss <= val_loss_min:
            print(
                f'Validation loss decreased ({val_loss_min:.5f} --> {val_loss:.5f})  Saving model to {model_file_name}...')
            torch.save(model.state_dict(), model_file_name)
            val_loss_min = val_loss

        print()

    # return model performance history
    return history

In [ ]:
# load the dataset
dataset_path = "build/Practicum/pics/2750"
transform = T.ToTensor()
dataset = ImageFolder(dataset_path, transform=transform)



dataset.class_to_idx
pp = pprint.PrettyPrinter(indent=1)  # Create a PrettyPrinter instance with an indentation of 1 space
pp.pprint(dataset.class_to_idx)  # Use the pprint method to print the dictionary

In [ ]:
# create the test, train and validation set and the dataloader

# setup split data 
dataset_size = len(dataset)
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15


# Set the seed for the generator
seed = 42
generator = torch.Generator().manual_seed(seed)

# Calculate the number of samples for each split
train_size = int(train_ratio * dataset_size)
val_size = int(val_ratio * dataset_size)
test_size = dataset_size - train_size - val_size

# Split the dataset into train, validation, and test sets
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size], generator=generator)


# Define the batch size for the DataLoader
batch_size = 128

# Create the DataLoader to load the dataset in batches
train_d = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_d = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_d = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

# Show the 1st img in the dataset
show_batch(train_d, test_d,val_d)
print(train_dataset[0][0])

In [ ]:
# define the CNN model

class CNN(nn.Module):
    
    #-------------------------------------------------------
    
    def __init__(self, dropout=0):
        self.dropout = dropout
        

        # Because we inherit from Module base class
        super().__init__()
        
        # RGB input, 6 filters, kernel of 5 x 5
        self.conv1 = nn.Conv2d(3, 6, 5)
        
        # Filter is 2 x 2 with a stride of 2 (defined once, used two times)
        self.pool = nn.MaxPool2d(2, 2)
        
        # in_channels = 6 because self.conv1 output has 6 channels
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        # Fully connected layer matched on output of conv2 layer
        self.fc1 = nn.Linear(16 * 13 * 13, 120)
        
        # Dropout layer1
        self.dropout1 = nn.Dropout(self.dropout)    

        # Dropout layer2
        self.dropout2 = nn.Dropout(self.dropout)

        # Fully connected layer
        self.fc2 = nn.Linear(120, 84)
        
        # We only have 2 labels
        self.fc3 = nn.Linear(84, 10)
        
    #-------------------------------------------------------
        
    def forward(self, x):
        
        # Convolution with relu layers
        x = self.pool(FF.relu(self.conv1(x)))
        x = self.pool(FF.relu(self.conv2(x)))
        
        # To match the output of the conv2 layer onto the first fully connected layer
        # Like reshape() but makes no copy (reuses underlaying data)
        x = x.view(-1, 16 * 13 * 13)
        
        # Fully connected layers
        x = FF.relu(self.fc1(x))
        x = self.dropout1(x)
        x = FF.relu(self.fc2(x))
        x = self.dropout2(x)
        
        # No activation on final layer 
        return self.fc3(x)


In [ ]:
# set the hyperparameters and the model

# hyperparameters
learning_rate = 0.01
epochs = 1
momentum = 0.9
total_epochs =0
dropout = 0.2


# model parameters

model = CNN(dropout=dropout) # Create the model
model.to(device) # set model to device
model.train() # set model to train mode

# optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)

# loss function
criterion = nn.CrossEntropyLoss()   # Define the loss function  
 
 # set the loaders to the correct names for the training loop
train_loader = train_d
val_loader = val_d

# set epochloss to empty list
epoch_loss = []

In [ ]:
# create dictionairies for the hyperparameters and model parameters
hyperparameters = { 'learning_rate' : learning_rate, 'epochs' : epochs, 'momentum' : momentum, 'dropout' : dropout}
model_parameters = { 'model' : 'cnn', 'optimizer' : 'SGD', 'criterion' : 'CrossEntropyLoss'}  # 'model' : model, 'optimizer' : optimizer, 'criterion' : criterion
parameters = {**hyperparameters, **model_parameters}    # merge the two dictionairies
pp = pprint.PrettyPrinter(indent=1)  

pp.pprint(f'hyperparameters: {hyperparameters}')
print()
pp.pprint(f'model_parameters: {model_parameters}')
print()
pp.pprint(f'parameters: {parameters}')

In [ ]:
# the training loop
timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
saveFileName = f'{timestamp}_{userAccountID}.pt'

history = train(
    model, train_loader, val_loader, criterion, optimizer, n_epochs=epochs,
    model_file_name=saveFileName
)

save the data to azure cosmos NoSQL database


In [ ]:

document = {'id': unique_id, 'pth_data': pth_bytes, 'hyperparameters': hyperparameters, 'model_parameters': model_parameters}
cf.save_data(saveFileName, parameters, history)



test


In [ ]:
# load the best model

from azure.cosmos import CosmosClient
import base64

import torch

# Retrieve settings from the config file
endpoint_uri = settings['host']
auth_key = settings['master_key']
database_name = settings['database_id']
container_name = settings['container_id']

# Create the Cosmos DB client
client = CosmosClient(endpoint_uri, auth_key)

# Get the database and container
database = client.get_database_client(database_name)
container = database.get_container_client(container_name)

# Retrieve the document containing the .pth data (assuming you know the unique_id)
unique_id = "20230602_0010_dyrr"
item = container.read_item(item=unique_id, partition_key=unique_id)

# Retrieve the pth_data from the item
pth_bytes = item['pth_data']

# Specify the path to save the .pth file
pth_file_path = f'/Users/stephandekker/workspace/pink_lady/noSQL/pth/imported/{unique_id}.pth'

# Decode and save the .pth data to a file
decoded_pth_data = base64.b64decode(pth_bytes)
with open(pth_file_path, 'wb') as file:
    file.write(decoded_pth_data)

# Retrieve the hyperparameters
hyperparameters = item['hyperparameters']

# Save the variable as a .pth file
torch.save(decoded_pth_data, pth_file_path)

# # print the hyperparameters
# print(hyperparameters)

In [ ]:
test_loader = test_d
len(test_loader)

In [ ]:
%%timeit
y_test = []
y_pred = []
counter = 0
# disable gradient calculation
with torch.no_grad():
    # prepare model for evaluation
    model.eval()

    # loop for each data
    for data, target in tqdm(test_loader):
        # move data to device
        data, target = data.to(device), target.to(device)

        counter += 1
        # STEP 1: forward pass
        output = model(data)
        # STEP 2: get predicted label
        _, label = torch.max(output, dim=1)
        # STEP 3: append actual and predicted label
        y_test += target.cpu().numpy().tolist()  # set to cpu and convert to list for old visualisation
        y_pred += label.cpu().numpy().tolist()
        print(counter)

print(f'End{counter}')

In [ ]:
# collapse-hide
plt.subplots(figsize=(10, 8))
ax = sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt="d")
ax.set_xlabel("Predicted Label")
ax.set_ylabel("Actual Label")
plt.show()

In [ ]:
print(classification_report(y_test, y_pred))